# Install Package

In [ ]:
# install pybaseball (for Google Colab)
!pip install pybaseball

# Import Packages

In [3]:
import numpy as np
import pandas as pd
import math
from pybaseball import statcast

# Extracting Statcast data (by using pybaseball)

In [ ]:
# Specify date(e.g. from 2024-05-01 to 2024-05-10)
pyb_df = statcast('2024-05-01', '2024-05-10')

# Limit to regular season data only
pyb_df = pyb_df[pyb_df["game_type"] == "R"]

In [ ]:
# Display the first 5 records of the dataframe
pyb_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
2618,CH,2024-05-10,90.1,-2.12,5.81,"Cano, Yennier",606466,666974,field_out,hit_into_play,...,1,4,2,1,1,1,2.95,1.22,-1.22,23.9
2670,CH,2024-05-10,90.4,-2.11,5.76,"Cano, Yennier",606466,666974,NaN,ball,...,1,4,2,1,1,1,2.77,1.41,-1.41,21.4
2731,CH,2024-05-10,91.8,-2.22,5.85,"Cano, Yennier",656976,666974,field_out,hit_into_play,...,1,0,2,1,1,3,2.53,1.25,-1.25,32.0
2829,CH,2024-05-10,90.4,-1.98,5.89,"Cano, Yennier",656976,666974,NaN,called_strike,...,1,0,2,1,1,3,2.51,1.41,-1.41,24.8
2952,CH,2024-05-10,90.3,-2.03,5.83,"Cano, Yennier",656976,666974,NaN,ball,...,1,0,2,1,1,3,3.0,1.56,-1.56,25.5


Calculating observed spin direction(tilt angle) and spin efficiency

In [5]:
'''
function to calculate vertical approach angle(vaa) and horizontal approach angle(haa)
'''
def vaa_and_haa(df):

    # vy_f = -sqrt(vy0^2 - 2*ay*(y0 - yf))
    # t = (yf - y0) / ay
    # vx_f = vx0 + ax * t
    # vz_f = vz0 + az * t
    # HAA = -arctan(vx_f / vy_f) * (180/pi)
    # VAA = -arctan(vz_f / vy_f) * (180/pi)

    y0 = 50.0       # determined at y=50 feet
    yf = 17/12      # home plate
    vy_f = -np.sqrt(df.vy0**2 - 2 * df.ay * (y0 - yf))
    t_final = (yf - y0) / df.ay
    vx_f = df.vx0 + df.ax * t_final
    vz_f = df.vz0 + df.az * t_final

    # Horizontal Approach Angle (HAA)
    haa = -np.arctan(vx_f / vy_f) * (180 / math.pi)

    # Vertical Approach Angle (VAA)
    vaa = -np.arctan(vz_f / vy_f) * (180 / math.pi)

    df['haa'] = haa
    df['vaa'] = vaa

    return df

In [ ]:
# Function Execution (add fields to dataframe)
pyb_df_vaa_haa = vaa_and_haa(pyb_df)

# Display the first 5 records of the dataframe
pyb_df_vaa_haa.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,haa,vaa
2589,CH,2024-05-10,90.1,-2.12,5.81,"Cano, Yennier",606466,666974,field_out,hit_into_play,...,2,1,1,1,2.95,1.22,-1.22,23.9,15.781713,25.106242
2672,CH,2024-05-10,90.4,-2.11,5.76,"Cano, Yennier",606466,666974,NaN,ball,...,2,1,1,1,2.77,1.41,-1.41,21.4,17.324474,21.811146
2771,CH,2024-05-10,91.8,-2.22,5.85,"Cano, Yennier",656976,666974,field_out,hit_into_play,...,2,1,1,3,2.53,1.25,-1.25,32.0,16.308115,19.788988
2860,CH,2024-05-10,90.4,-1.98,5.89,"Cano, Yennier",656976,666974,NaN,called_strike,...,2,1,1,3,2.51,1.41,-1.41,24.8,16.121592,18.234036
2877,CH,2024-05-10,90.3,-2.03,5.83,"Cano, Yennier",656976,666974,NaN,ball,...,2,1,1,3,3.0,1.56,-1.56,25.5,19.479676,22.221033
